## Load libraries

In [ ]:
using HTTP
using JSON
using DataStructures

## Set token/URL

In [ ]:
token = get(ENV, "API_TOKEN", "")
@assert !isempty(token)
url = get(ENV, "API_BASE_URL", "https://covid-modelling-stg.epcc.ed.ac.uk")

In [ ]:
model_slug = "sir-ode-python"
headers = OrderedDict(
    "accept"=>"application/json; charset=utf-8", 
    "Content-Type"=>"application/json", 
    "Authorization"=> "Bearer "*token
)

## Input

In [ ]:
inputJSON = OrderedDict(
  "p" => [
    0.5,
    0.25
  ],
  "u0" => [
    0.99,
    0.01,
    0
  ],
  "tspan" => [
    0,
    10000
  ]
)

## Post simulations

In [ ]:
post_simulations_url = url * "/api/simulations/model-runs/" * model_slug

In [ ]:
post_simulations_response = HTTP.post(post_simulations_url, headers, json(inputJSON))

@assert HTTP.status(post_simulations_response) == 200

post_simulations_content = JSON.parse(String(HTTP.body(post_simulations_response)))

We need the ID to construct other URLs later.

In [ ]:
simulation_id = string(post_simulations_content["id"])

## Check simulations

In [ ]:
check_simulations_url = url * "/api/simulations/" * simulation_id * "/model-runs/" * model_slug

In [ ]:
check_simulations_response = HTTP.get(check_simulations_url, headers)

@assert HTTP.status(check_simulations_response) == 200

check_simulations_content = JSON.parse(String(HTTP.body(check_simulations_response)))

The process can take some time to complete.
The following code polls the API to check for success every `step` seconds, with a `timeout` in seconds.

In [ ]:
stp = 60
timeout = 600

tm = 0
check_simulations_response = nothing
while tm <= timeout
    check_simulations_response = HTTP.get(check_simulations_url, headers)
    check_simulations_content = JSON.parse(String(HTTP.body(check_simulations_response)))
    if check_simulations_content["status"] == "complete"
        break
    end
    sleep(stp)
    tm += stp
end

In [ ]:
check_simulations_content

## Export simulation output

In [ ]:
export_simulations_url =  url * "/api/simulations/" * simulation_id * "/model-runs/" * model_slug * "/export"

In [ ]:
export_simulations_response = HTTP.get(export_simulations_url, headers)

@assert HTTP.status(export_simulations_response) == 200

export_simulations_content = JSON.parse(String(HTTP.body(export_simulations_response)))